<a href="https://colab.research.google.com/github/biglool/Hologram_Cell_detection/blob/main/notebooks/backpro_ejemplo/Backpropagacion_holograms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


In [ ]:
!git clone -l -s "https://github.com/biglool/Hologram_Cell_detection/" holo_cells

In [ ]:

import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import json

from holo_cells.src.Generador_datos.unxip_auxiliars import apodize,reconstruct_hologram_with_twin_image_removal

carpeta_imatges= "/content/gdrive/MyDrive/Curro/Hologrames/Datos_reales/2024_01_25/" # aixo es el meu drive
# !!!!! la carpeta la he compartit a https://drive.google.com/drive/folders/12KfNBywcqimDPgFHU-Dqj32LZ858tY4a?usp=drive_link
# !!!!! Podeu en principi accedir desde el drive com faig jo, pero heu de fer: https://stackoverflow.com/questions/54351852/accessing-shared-with-me-with-colab i corretgir la ruta


#!!!!! notar que aqui es pillen mosaics, les imatges soles haurien de funcionar ,
#!!!!! pero quan feu la captura pilleu nomes una grid perque us guardi la info del mount correctament( sino ho mirem)
mostres=["Samples_01_normalized_mosaic", "Samples_03_2_normalized_mosaic",]


capture_file = carpeta_imatges+'capture_collection.json'

with open(capture_file, 'r') as file:
    capture_data = json.load(file)

def find_annotations_for_image(image_uid, analysis_data):
      for key, value in analysis_data.items():
          if 'iid' in value and value['iid'] == image_uid:
              return value.get('mount', [])
      return []

for imagen_name in mostres:

  if os.path.exists(carpeta_imatges+imagen_name):
    for f in os.listdir(carpeta_imatges+imagen_name):
      print(carpeta_imatges+imagen_name+f)
      #mount info
      mount=find_annotations_for_image(imagen_name, capture_data)
      im=Image.open(carpeta_imatges+imagen_name+"/"+f)
      l=mount["wavelength"]* 1e-9
      h=im.size[1] * mount["pitch_camera"]* 1e-6
      z=mount["distance_display_camera"]* 1e-3
      z0=mount["distance_display_sample"]* 1e-6
      twin_image_iterations=5
      apodization=118

      #visualzacio original y projecció
      plt.imshow(np.array(im))
      plt.show()
      im=apodize(im,int((apodization/4)*3))
      proyeccion= np.array(reconstruct_hologram_with_twin_image_removal(im,l,h,z,z0))
      plt.imshow(proyeccion)
      plt.show()